# 预分析政策法规

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data_path = "./政策法规.parquet"
df = pd.read_parquet(data_path)

In [ ]:
df = df.sort_values("publish_date", ascending=False)

In [ ]:
df

In [ ]:
# 统计df的content_markdown的长度分布
df["content_markdown_len"] = df["content_markdown"].apply(lambda x: len(x))
df["content_markdown_len"].hist()

In [ ]:
import numpy as np


bins = np.arange(0, 10000 + 200, 2000)  # 生成0, 200, 400,..., max_length的分组
 
# 使用cut函数分组并统计频数
distribution = pd.cut(df['content_markdown_len'], bins=bins, right=False).value_counts().sort_index()
 
# 转换为更友好的区间标签
distribution.index = distribution.index.astype(str)
result = distribution.reset_index()
result.columns = ['字数区间', '数量']
 
# 打印结果
print(result)

In [ ]:
# 设置中文字体，以防绘图时出现乱码
plt.rcParams['font.sans-serif'] = ['STHeiti']
plt.rcParams['axes.unicode_minus'] = False

## 角度一：元数据统计分析
1. 目的
通过分析政策的发布时间、发布机构、法规类别等元数据，可以展示数据集的广度和多样性。一个覆盖时间长、来源广泛、类别多样的数据集，意味着其潜在的知识图谱将具有更高的研究和应用价值。

In [ ]:
# --- 1. 发布年份分布分析 ---
print("--- 1. 发布年份分布分析 ---")
# 将日期列转换为datetime对象，errors='coerce'会把无法转换的设为NaT
df['publish_date'] = pd.to_datetime(df['publish_date'], errors='coerce')
df['publish_year'] = df['publish_date'].dt.year

# 过滤掉无效年份
year_counts = df['publish_year'].dropna().astype(int).value_counts().sort_index()

plt.figure(figsize=(12, 6))
sns.barplot(x=year_counts.index, y=year_counts.values, palette="viridis")
plt.title('政策文件发布年份分布', fontsize=16)
plt.xlabel('年份', fontsize=12)
plt.ylabel('文件数量', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

publish_year,count
2007.0,642
2006.0,607
2008.0,557
2013.0,552
2012.0,538
2011.0,533
2004.0,524
2016.0,519
2005.0,512
2021.0,500
2010.0,468
2020.0,466
2018.0,455
2009.0,443
2015.0,434
2014.0,416
2019.0,412
2003.0,408
2017.0,385
2022.0,375
2023.0,346
2001.0,316
2002.0,308
2024.0,303
2000.0,289
2025.0,262


In [ ]:
print(df['publish_year'].value_counts().to_csv())

In [ ]:

# --- 2. 发布机构TOP 10分析 ---
print("\n--- 2. 发布机构TOP 10分析 ---")
authority_counts = df['issuing_authority'].value_counts().nlargest(20)

plt.figure(figsize=(12, 6))
sns.barplot(x=authority_counts.values, y=authority_counts.index, orient='h', palette="plasma")
plt.title('TOP 10 政策发布机构', fontsize=16)
plt.xlabel('文件数量', fontsize=12)
plt.ylabel('发布机构', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
print(authority_counts.to_csv())

In [ ]:
# --- 3. 法规类别分布分析 ---
print("\n--- 3. 法规类别分布分析 ---")
category_counts = df['category'].value_counts().nlargest(10)
plt.figure(figsize=(10, 10))
plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("Set2"))
plt.title('政策法规类别分布 (TOP 10)', fontsize=16)
plt.axis('equal') # 保证饼图是圆的
plt.tight_layout()
plt.show()


In [ ]:
print(category_counts.to_csv())

## 角度二：文本内容基础指标分析
1. 目的
通过计算每篇政策文件的文本长度，我们可以评估文档的信息密度。如果大部分文件都具有相当的长度，而不是寥寥数语的通知，那么就更有可能从中抽取出有价值的实体和复杂的关系。

In [ ]:
import numpy as np

# --- 4. 政策内容长度分析 ---
print("\n--- 4. 政策内容长度分析 ---")

# 确保content_markdown列是字符串类型，并填充空值为''
df['content_markdown'] = df['content_markdown'].astype(str).fillna('')
df['content_length'] = df['content_markdown'].apply(len)

# 移除长度为0的无效数据以获得更准确的分析
valid_lengths = df[df['content_length'] > 0]['content_length']

# 计算统计数据
avg_len = valid_lengths.mean()
median_len = valid_lengths.median()
max_len = valid_lengths.max()
min_len = valid_lengths.min()

print(f"有效政策文本的平均长度: {avg_len:.2f} 字符")
print(f"有效政策文本的长度中位数: {median_len} 字符")
print(f"最长文本长度: {max_len} 字符")
print(f"最短文本长度: {min_len} 字符")

# 绘制文本长度的直方图和箱线图
plt.figure(figsize=(12, 6))
sns.histplot(valid_lengths, bins=50, kde=True)
plt.title('政策文件内容长度分布', fontsize=16)
plt.xlabel('字符数', fontsize=12)
plt.ylabel('文件数量', fontsize=12)
# 使用对数刻度可以更好地观察长尾分布
plt.xscale('log')
plt.tight_layout()
plt.show()

你可以展示文本长度的平均值、中位数和分布直方图，并论述：“数据集中文本的平均长度达到了XXXX字符，且从分布图来看，绝大多数文件都包含丰富的内容，而非简短公告。这表明文本中蕴含了足量的细节信息，为深入的实体识别和关系抽取提供了坚实的基础。”

## 角度三：文本内容主题分析
1. 目的
使用自然语言处理（NLP）技术，如TF-IDF和主题模型（LDA），可以自动发现文本数据中的核心议题和隐藏主题。这能直观地展示数据内容的聚合性和发散性，证明文本中存在可供提取的、有意义的“知识点”。

In [ ]:
import jieba
from collections import Counter
words_counts = Counter()
_ = df["content_markdown"].apply(lambda content: words_counts.update(list(jieba.cut(content))),)

In [ ]:
# 去除非中文和单字
import re
# 定义正则表达式：匹配长度≥2的纯中文字符
pattern = re.compile(r'^[\u4e00-\u9fff]{2,}$')
 
# 过滤 Counter，保留符合条件的键
filtered_counts = Counter()
for word, count in words_counts.items():
    if pattern.match(word):
        filtered_counts[word] = count
 
# 直接覆盖原 Counter（或使用新变量）
words_counts = filtered_counts

In [ ]:
words_counts.most_common(10)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

font_path = "/System/Library/Fonts/STHeiti Light.ttc" # 请确保路径正确

wordcloud_log = WordCloud(
    font_path=font_path,
    background_color="white",
    width=2400,
    height=1697,
    max_words=5000,
    #colormap='gray',
    relative_scaling=0.2,         # 使用对数词频后，可以适当调高此参数
    max_font_size=300,
    min_font_size=10,
    scale=2
).generate_from_frequencies(words_counts)

plt.figure(figsize=(8.27, 5.83), dpi=300) # A4 尺寸 (英寸) 和打印级DPI
plt.imshow(wordcloud_log, interpolation="bilinear") # 决定使用哪个方案的词云
plt.axis("off")                                  # 隐藏坐标轴
plt.tight_layout(pad=0)                          # 去除图像周围的白边
plt.show()